In [ ]:
def sample_qry(conn):
    cursor=conn.cursor()
    q=f'''
      select * from ins_crdif_inq_agr_adm 
        order by inp_dthms desc 
        limit 1
      '''
    cursor.execute(q)
    for row in cursor:
        print(row)
#     conn.close()

import pyodbc
connection_string = (
    'DRIVER=MySQL ODBC 8.0 ANSI Driver;'
    'SERVER=10.149.***.**,****;'
    'DATABASE=k***;'
    'UID=******;'
    'PWD=******;'
#     'charset=utf8;'
    'MULTI_HOST=1;'
)
conn=pyodbc.connect(connection_string)
# conn.setencoding(encoding='utf-8')
# sample_qry(conn)
print('completed..')

In [15]:
import pandas as pd
tbl='ins_cr_scp_rt'
# tbl='ins_crdif_inq_agr_adm'
path='c:/kpayins/temp/'+tbl+'_out.xlsx'
# df=pd.read_excel('c:/kpayins/temp/ins_crdif_inq_agr_adm.xlsx',sheet_name='data',header=0,dtype='object')
df=pd.read_excel(path,sheet_name='data',header=0,dtype='object',na_filter=False)

In [16]:
def make_sql(tbl,row):
    #값이 있는 컬럼의 index to list
    non_empty=[i for i in range(0,len(row)) if row[i] !=""]
    #값이 있는 컬럼명만 추출 to list
    cols=[str(x) for x in row.index[non_empty]]
    #값만 추출 to list
    params=[str(row[i]) for i in non_empty]
#     print(len(cols),len(params))
    q=f'''
      insert into {tbl}
      ( {",".join([str(i) for i in cols])} )
      values 
      ( {",".join(["?" for i in params])} )
     '''
    q= q.replace('\n','')
#     print(q)
    return q,params

cursor=conn.cursor()
for i, row in df.iterrows():
    q,params= make_sql(tbl,row)
    cursor.execute(q, params)
    cursor.commit()
#     print(q,params)
cursor.close()
print('completed...')

completed...


## 이하는 참고용

In [ ]:
# bulk insert 1
for row_count in range(0, df_op.shape[0]):
      chunk = df_op.iloc[row_count:row_count + 1,:].values.tolist()
      tuple_of_tuples = tuple(tuple(x) for x in chunk)
      cursor.executemany("insert into test" + " ([col_name1],   col_name2],[col_name3],[col_name4],[col_name5],[col_name6],[col_name7],[col_name8],[col_name9],[col_name10]) values   (?,?,?,?,?,?,?,?,?,?)",tuple_of_tuples)

In [ ]:
# bulk insert 2
cursor.fast_executemany = True
for row_count in range(0, df_op.shape[0]):
   chunk = df_op.iloc[row_count:row_count + 1,:].values.tolist()
   tuple_of_tuples = tuple(tuple(x) for x in chunk)
   cursor.executemany("insert into test" + " ([col_name1],   col_name2],[col_name3],[col_name4],[col_name5],[col_name6],[col_name7],[col_name8],[col_name9],[col_name10]) values   (?,?,?,?,?,?,?,?,?,?)",tuple_of_tuples)

In [ ]:
# bulk insert 3
import urllib,sqlalchemy
db_params = urllib.parse.quote_plus(connection_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
#df is the dataframe; test is table name in which this dataframe is #inserted
# df.to_sql('ins_crdif_inq_agr_adm,engine',engine,index=False,if_exists="append",schema="dbo")
dt=df.T
dt.to_sql('ins_crdif_inq_agr_adm',con=engine,index=True)

In [ ]:
df.to_sql?

In [ ]:
# bulk insert 4 : 가장 빠름
from sqlalchemy import event
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
       if executemany:
            cursor.fast_executemany = True

df.to_sql(tbl, engine, index=False, if_exists="append", schema="dbo")